In [18]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from dotenv import dotenv_values
import pandas as pd
import time

In [19]:
# Variablen für den Lauf
cenv = dotenv_values(".env")

#mail_to = "jens-ole@petersen.top"
mail_subject = "IGRM: Rechnung Mitgliedsbeitrag 2021"
attachment_folder = "out/"

In [20]:
def send_email(to_address, mitgliedsnummer):

    mail_attachment = attachment_folder + "IGRM_Mitgliedsrechnung_" + str(mitgliedsnummer) + ".pdf"
    mail_attachment_name = "IGRM_Mitgliedsrechnung_" + str(mitgliedsnummer) + ".pdf"

    #mail_attachment = "C:/Coding/IGRM-PDF-Invoice-Generation/IGRM-PDF-Invoice-Generation/out/IGRM_Mitgliedsrechnung_4711.pdf"

    mimemsg = MIMEMultipart('alternative')
    mimemsg['From'] = cenv["mail_from"]
    mimemsg['To'] = to_address
    mimemsg['Subject'] = mail_subject

    with open("mail-template.txt") as f:
        bodytext = f.read()
    with open("mail-template.html") as f:
        bodyhtml = f.read()

    mimemsg.attach(MIMEText(bodytext, 'plain'))
    mimemsg.attach(MIMEText(bodyhtml, 'html'))

    with open(mail_attachment, "rb") as attachment:
        mimefile = MIMEBase('application', 'octet-stream')
        mimefile.set_payload((attachment).read())
        encoders.encode_base64(mimefile)
        mimefile.add_header('Content-Disposition', "attachment; filename= %s" % mail_attachment_name)
        mimemsg.attach(mimefile)

        connection = smtplib.SMTP(host=cenv["host"], port=cenv["port"])
        connection.starttls()
        connection.login(cenv["username"], cenv["password"])
        connection.send_message(mimemsg)
        connection.quit()

In [21]:
# Einlesen der Daten für den Lauf
liste = pd.read_excel("Rechnungsversand 2021.xlsx")    #csv: , sep=";"
rm_quote = lambda x: x.replace('"', '')
liste = liste.rename(columns=rm_quote)
liste = liste.set_index("ID")

In [23]:
for i in range(len(liste)):

    d = liste.iloc[i]

    mgnr = d["Mitgliedsnummer"] 
    st = d["E-Mail Rechnungsversand"]
    print(mgnr, st)
    st = "block schutz"
    send_email(st, mgnr)

    time.sleep(30)  # Send Limit in O365 of 30 Mail per minute

1000 bernpayables@cslbehring.com


SMTPRecipientsRefused: {'block': (501, b'5.1.3 Invalid address')}